In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
import os 
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key = api_key
)

In [5]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [7]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [9]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [10]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker always have plenty of money?\n\nBecause he knew how to make a lot of **dough**!',
 'explanation': 'This joke is a classic **pun**, playing on the word "dough" having two different meanings:\n\n1.  **Literal Meaning (related to pizza making):** "Dough" is the mixture of flour, water, and other ingredients that you use to make bread, pizza crust, pastries, etc. A pizza maker literally "makes a lot of dough" in their job.\n2.  **Slang Meaning:** "Dough" is a common informal term for **money**.\n\nThe humor comes from the setup asking about the pizza maker having "plenty of money." The punchline then uses "dough" in a way that *seems* to refer to the literal pizza ingredient, but cleverly implies the slang meaning of "money," providing a funny and unexpected answer to the question.\n\nSo, he has plenty of money because he knows how to make a lot of **money** (dough)!'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker always have plenty of money?\n\nBecause he knew how to make a lot of **dough**!', 'explanation': 'This joke is a classic **pun**, playing on the word "dough" having two different meanings:\n\n1.  **Literal Meaning (related to pizza making):** "Dough" is the mixture of flour, water, and other ingredients that you use to make bread, pizza crust, pastries, etc. A pizza maker literally "makes a lot of dough" in their job.\n2.  **Slang Meaning:** "Dough" is a common informal term for **money**.\n\nThe humor comes from the setup asking about the pizza maker having "plenty of money." The punchline then uses "dough" in a way that *seems* to refer to the literal pizza ingredient, but cleverly implies the slang meaning of "money," providing a funny and unexpected answer to the question.\n\nSo, he has plenty of money because he knows how to make a lot of **money** (dough)!'}, next=(), config={'configurable': {'thread_id': '1

In [12]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker always have plenty of money?\n\nBecause he knew how to make a lot of **dough**!', 'explanation': 'This joke is a classic **pun**, playing on the word "dough" having two different meanings:\n\n1.  **Literal Meaning (related to pizza making):** "Dough" is the mixture of flour, water, and other ingredients that you use to make bread, pizza crust, pastries, etc. A pizza maker literally "makes a lot of dough" in their job.\n2.  **Slang Meaning:** "Dough" is a common informal term for **money**.\n\nThe humor comes from the setup asking about the pizza maker having "plenty of money." The punchline then uses "dough" in a way that *seems* to refer to the literal pizza ingredient, but cleverly implies the slang meaning of "money," providing a funny and unexpected answer to the question.\n\nSo, he has plenty of money because he knows how to make a lot of **money** (dough)!'}, next=(), config={'configurable': {'thread_id': '

In [12]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was tired of feeling so **saucy** all the time!',
 'explanation': 'This joke is a pun that plays on the word "**saucy**" and the personification of pasta.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Food):** When pasta is mixed with sauce, it is literally "saucy" – meaning it\'s covered in or full of sauce. This is the initial, surface-level understanding the joke sets up.\n\n2.  **Figurative Meaning (Personality):** "Saucy" also has a common figurative meaning when describing a person\'s behavior. It means to be:\n    *   **Cheeky or disrespectful** in a lively or impudent way.\n    *   **Bold, sassy, or impertinent.**\n    *   Sometimes flirtatious or suggestive.\n\n3.  **The Punchline:** The humor comes from the joke giving the pasta human emotions and intentions (personification). The pasta "breaks up" with the sauce because it\'s tired of feeling "saucy" – not just literally covered in sa

In [ ]:
workflow.get_state(config2)

In [14]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was tired of feeling so **saucy** all the time!', 'explanation': 'This joke is a pun that plays on the word "**saucy**" and the personification of pasta.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Food):** When pasta is mixed with sauce, it is literally "saucy" – meaning it\'s covered in or full of sauce. This is the initial, surface-level understanding the joke sets up.\n\n2.  **Figurative Meaning (Personality):** "Saucy" also has a common figurative meaning when describing a person\'s behavior. It means to be:\n    *   **Cheeky or disrespectful** in a lively or impudent way.\n    *   **Bold, sassy, or impertinent.**\n    *   Sometimes flirtatious or suggestive.\n\n3.  **The Punchline:** The humor comes from the joke giving the pasta human emotions and intentions (personification). The pasta "breaks up" with the sauce because it\'s tired of feeling "saucy" – not just lit

# Fault Tolerence

In [4]:

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [5]:
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [6]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [7]:
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

# Time Travel

In [ ]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

In [ ]:
list(workflow.get_state_history(config1))

# Updating State

In [ ]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

In [ ]:
list(workflow.get_state_history(config1))

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})


In [ ]:
list(workflow.get_state_history(config1))


# The End, not from Learning